Author: ProtonX Team

Website: https://protonx.ai/

## 1. Load dữ liệu

In [1]:
# Hướng dẫn về bộ dữ liệu này được thực hiện trong Lab Tokenizer 
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json  
import json

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

/bin/bash: /home/danieldu/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
wget: /home/danieldu/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-10-09 18:23:48--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.43.27, 142.251.42.251, 172.217.163.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.43.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.07s   

2023-10-09 18:23:48 (82.6 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



## 2. Tách từ

In [2]:
dataset = []
label_dataset = []
for item in datastore:
    dataset.append(item["headline"])
    label_dataset.append(item["is_sarcastic"])

In [3]:
dataset[:10], label_dataset[:10]

(["former versace store clerk sues over secret 'black code' for minority shoppers",
  "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  "mom starting to fear son's web series closest thing she will have to grandchild",
  'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  'j.k. rowling wishes snape happy birthday in the most magical way',
  "advancing the world's women",
  'the fascinating case for eating lab-grown meat',
  'this ceo will send your kids to school, if you work for his company',
  'top snake handler leaves sinking huckabee campaign',
  "friday's morning email: inside trump's presser for the ages"],
 [0, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [4]:
import numpy as np

dataset = np.array(dataset)
label_dataset = np.array(label_dataset)

In [5]:
dataset

array(["former versace store clerk sues over secret 'black code' for minority shoppers",
       "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
       "mom starting to fear son's web series closest thing she will have to grandchild",
       ..., 'reparations and obama',
       'israeli ban targeting boycott supporters raises alarm abroad',
       'gourmet gifts for the foodie 2014'], dtype='<U254')

Chia dữ liệu

In [6]:
train_size = 0.8
size = int(len(dataset) * train_size)

train_sentence = dataset[:size]
test_sentence = dataset[size:]

train_label = label_dataset[:size]
test_label = label_dataset[size:]

In [7]:
len(train_sentence), len(test_sentence)

(21367, 5342)

### 2.1. Train_sequences

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-10-09 18:23:49.282159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
vocab_size = len(train_sentence)
embedding_size = 64
max_length = 25

In [10]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentence)

In [11]:
tokenizer.word_index

{'<OOV>': 1,
 'to': 2,
 'of': 3,
 'the': 4,
 'in': 5,
 'for': 6,
 'a': 7,
 'on': 8,
 'and': 9,
 'with': 10,
 'is': 11,
 'new': 12,
 'trump': 13,
 'man': 14,
 'from': 15,
 'at': 16,
 'about': 17,
 'you': 18,
 'by': 19,
 'this': 20,
 'after': 21,
 'up': 22,
 'out': 23,
 'be': 24,
 'how': 25,
 'that': 26,
 'it': 27,
 'as': 28,
 'not': 29,
 'are': 30,
 'your': 31,
 'what': 32,
 'his': 33,
 'all': 34,
 'he': 35,
 'who': 36,
 'just': 37,
 'has': 38,
 'will': 39,
 'more': 40,
 'into': 41,
 'one': 42,
 'year': 43,
 'report': 44,
 'have': 45,
 'over': 46,
 'area': 47,
 'why': 48,
 'donald': 49,
 'u': 50,
 'day': 51,
 'can': 52,
 'says': 53,
 's': 54,
 'first': 55,
 'woman': 56,
 'time': 57,
 'like': 58,
 'get': 59,
 'her': 60,
 'old': 61,
 "trump's": 62,
 'no': 63,
 'now': 64,
 'off': 65,
 'an': 66,
 'life': 67,
 'obama': 68,
 'people': 69,
 'than': 70,
 'women': 71,
 "'": 72,
 'house': 73,
 'back': 74,
 'was': 75,
 'still': 76,
 'white': 77,
 'make': 78,
 'down': 79,
 'clinton': 80,
 'my': 81,

In [12]:
train_sequences = tokenizer.texts_to_sequences(train_sentence)

In [13]:
train_sentence[1]

"the 'roseanne' revival catches up to our thorny political mood, for better and worse"

In [14]:
train_sequences[1]

[4, 7191, 2989, 2990, 22, 2, 154, 9271, 388, 2751, 6, 265, 9, 965]

In [15]:
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating="post", padding="post")

In [16]:
max_length

25

In [17]:
padded_train_sequences[2:4]

array([[ 156,  924,    2,  865, 1530, 2097,  599, 5049,  220,  135,   39,
          45,    2, 9272,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0],
       [1352,   37,  218,  382,    2, 1680,   29,  294,   22,   10, 2359,
        1416, 5903, 1004,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0]], dtype=int32)

In [18]:
padded_train_sequences.shape

(21367, 25)

### 2.2. Test_sequences

In [19]:
test_sequences = tokenizer.texts_to_sequences(test_sentence)

In [20]:
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating="post", padding="post")

In [21]:
padded_test_sequences.shape

(5342, 25)

Custom Model LSTM

In [22]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Lập trình thuật toán cho một cell LSTM bao gồm các cổng đề điều chỉnh luồng thông tin:
- Cổng kiểm soát đầu vào
- Cổng kiểm soát số lượng dữ liệu giữ lại/quên đi
- Cổng kiểm soát dữ liệu đầu ra

Code dưới đây kế thừa class Layer từ keras để override một số hàm cho mô hình cụ thể của chúng ta.

[Xem thêm](https://www.tensorflow.org/tutorials/customization/custom_layers) cách sử dụng tại.

In [23]:
class LSTM(tf.keras.layers.Layer):
  def __init__(self, units, inp_shape):
    super(LSTM, self).__init__()
    self.units = units
    self.inp_shape = inp_shape
    self.W = self.add_weight("W", shape=(4, self.units, self.inp_shape))
    self.U = self.add_weight("U", shape=(4, self.units, self.units))


  def call(self, pre_layer, x):
    pre_h, pre_c = tf.unstack(pre_layer)
    
    # Cổng kiểm soát đầu vào: Input Gate

    i_t = tf.nn.sigmoid(tf.matmul(x, tf.transpose(self.W[0])) + tf.matmul( pre_h, tf.transpose(self.U[0])))

    # Cổng kiểm soát số lượng dữ liệu giữ lại/quên đi: Forget Gate
    f_t = tf.nn.sigmoid(tf.matmul(x, tf.transpose(self.W[1])) + tf.matmul( pre_h, tf.transpose(self.U[1])))

    # Cổng kiểm soát dữ liệu đầu ra: Output Gate
    o_t = tf.nn.sigmoid(tf.matmul(x, tf.transpose(self.W[2])) + tf.matmul( pre_h, tf.transpose(self.U[2])))

    # Đây giống SimpleRNN và được coi là thông tin mới (bộ nhớ mới)

    n_c_t = tf.nn.tanh(tf.matmul(x, tf.transpose(self.W[3])) + tf.matmul( pre_h, tf.transpose(self.U[3])))

    # Kết hợp việc giữ lại thông tin + bổ sung thêm thông tin mới

    c = tf.multiply(f_t, pre_c) + tf.multiply(i_t, n_c_t)

    # Cho phép bao nhiêu thông tin thoát khỏi cell

    h = tf.multiply(o_t, tf.nn.tanh(c))
    
    return tf.stack([h, c])


## 3. Xây dựng mô hình 

Thừa kế lại lớp Model của Keras để xây dựng mô hình riêng cho mình. 



Mô hình bao gồm
- Embedding để chuyển câu thành các vector với chiều `embedding_size`
- Sử dụng cell LSTM đã lập trình bên trên 
- Sau đó đưa lịch sử `h` của LSTM qua mạng nơ ron đơn giản

Qúa trình training bao gồm
- Đưa câu qua Embedding để lấy các vector
- Đưa `tuần tự` từng từ qua LSTM + (hidden_state và cell_state) lớp phía trước để thu được (hidden_state và cell_state) hiện tại
- Sử dụng hidden_state cuối cùng cho việc dự đoán

In [24]:
import numpy as np

In [25]:
class Bidirectional(tf.keras.Model):
  def __init__(self, lstm1, lstm2):
    super(Bidirectional, self).__init__()
    self.lstm1 = lstm1
    self.lstm2 = lstm2

  def call(self):
    # Left to right


    # Right to left
    pass

In [26]:
class ProtonXRNN(tf.keras.Model):
  def __init__(self, units, embedding_size, vocab_size, input_length):
    super(ProtonXRNN, self).__init__()
    self.input_length = input_length
    self.units = units

    # Embedding để chuyển từ thành vector 
    self.embedding = tf.keras.layers.Embedding(
      vocab_size,
      embedding_size,
      input_length = input_length
    )

    # Sử dụng cell LSTM đã lập trình bên trên
    self.lstm = LSTM(units, embedding_size)

    # Sau đó đưa lịch sử h của LSTM qua mạng nơ ron đơn giản    
    self.classfication_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(units,)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    
  def call(self, sentence):
    """
    Parameters:
    sentence:
      Dạng: Tensor
      Miêu tả: Câu
      Chiều: (batch_size, input_length)
    out:
      Dạng: Tensor
      Miêu tả: Đầu ra của mô hình dự đoán
      Chiều: (batch_size, 1)
    """

    batch_size = tf.shape(sentence)[0]

    # Khởi tạo (hidden_state và context_state)

    pre_layer = tf.stack([
      tf.zeros([batch_size, self.units]), 
      tf.zeros([batch_size, self.units])
    ])

    # Đưa câu qua Embedding để lấy các vector
    # embedded_sentence: (batch_size, input_length, embedding_size)
    embedded_sentence = self.embedding(sentence)

    # Đưa tuần tự từng từ qua LSTM + (hidden_state và cell_state) 
    # lớp phía trước để thu được (hidden_state và cell_state) hiện tại
    for i in range(self.input_length):
      # : đầu tiên: Lấy batch_size
      # i Vị trí từ
      # : cuối cùng: Lấy embedding.
      # (batch_size, embedding_size)
      word = embedded_sentence[:, i, :]
      pre_layer = self.lstm(pre_layer, word)

    
    h, _ = tf.unstack(pre_layer)

    # Sử dụng hidden_state cuối cùng cho việc dự đoán
    return self.classfication_model(h)

units = 128
embedding_size = 100
vocab_size = len(tokenizer.index_word) + 1
input_length = max_length

# Khởi tạo đối tượng protonxrnn
protonxrnn = ProtonXRNN(units, embedding_size, vocab_size, input_length)


protonxrnn.compile(
    tf.keras.optimizers.Adam(0.0005) , loss='binary_crossentropy', metrics=['acc']
)
# Tiến hành training 
protonxrnn(padded_train_sequences[10:13]).shape

2023-10-09 18:23:51.503032: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


TensorShape([3, 1])

In [ ]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# import tensorflow as tf
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## 4. Tiến hành training

In [28]:
with tf.device(tf.DeviceSpec(device_type="GPU", device_index=0)):
    protonxrnn.fit(padded_train_sequences, train_label, validation_data=(padded_test_sequences, test_label) ,batch_size=32, epochs=100)

Epoch 1/100


668/668 [==============================] - 41s 52ms/step - loss: 0.5654 - acc: 0.6733 - val_loss: 0.3708 - val_acc: 0.8383
Epoch 2/100
668/668 [==============================] - 37s 56ms/step - loss: 0.2653 - acc: 0.8937 - val_loss: 0.3423 - val_acc: 0.8502
Epoch 3/100
668/668 [==============================] - 36s 54ms/step - loss: 0.1452 - acc: 0.9473 - val_loss: 0.3637 - val_acc: 0.8484
Epoch 4/100
668/668 [==============================] - 37s 55ms/step - loss: 0.0854 - acc: 0.9707 - val_loss: 0.4497 - val_acc: 0.8401
Epoch 5/100
668/668 [==============================] - 37s 56ms/step - loss: 0.0529 - acc: 0.9828 - val_loss: 0.5648 - val_acc: 0.8347
Epoch 6/100
668/668 [==============================] - 37s 55ms/step - loss: 0.0359 - acc: 0.9887 - val_loss: 0.7631 - val_acc: 0.8360
Epoch 7/100
668/668 [==============================] - 36s 54ms/step - loss: 0.0242 - acc: 0.9929 - val_loss: 0.7231 - val_acc: 0.8300
Epoch 8/100
668/668 [==============================] - 36s 54ms/ste